# 📣  利用 RAG 建立自己的查詢系統

## 🎧 Spotify 台灣流行音樂資料抓取

### 本次建立自己的檢索系統，使用之範例料為 Spotify 音樂播放清單之資料。包含播放清單類型與屬性，以及內含歌曲的基本資料（歌手、專輯名稱、專輯發布日期等）。

資料抓取流程：
- 利用 Spotify Developer 的 API 抓取不同關鍵字組合下能搜尋到的播放清單

- 抓取清單內每首歌的：曲名 / 藝人 / 專輯 / 發行日期 / Spotify 連結

- 匯出 .txt 作為 RAG 音樂知識庫素材

- 共抓取了 117159 首歌曲資料

### 0. 讀入檔案
以下將使用 `gdown` 下載已建立好的zip檔案，也就是包含向量資料庫的檔案

內涵有所儲存的歌曲資料

In [1]:
%pip install gdown

import gdown

file_id = "1PJmeVzBnkkJiAfgopZPu9UyZu3JpOmlY"

# 用gdown下載，檔名叫 faiss_db.zip
gdown.download(f"https://drive.google.com/uc?id={file_id}", "faiss_db.zip", quiet=False)

Note: you may need to restart the kernel to use updated packages.


Downloading...
From (original): https://drive.google.com/uc?id=1PJmeVzBnkkJiAfgopZPu9UyZu3JpOmlY
From (redirected): https://drive.google.com/uc?id=1PJmeVzBnkkJiAfgopZPu9UyZu3JpOmlY&confirm=t&uuid=b52dbc34-ab29-44f4-810b-be6b384de58d
To: /Users/andrewhsu/Documents/Generative_AI/7_RAG/faiss_db.zip
100%|██████████| 32.5M/32.5M [00:04<00:00, 6.55MB/s]


'faiss_db.zip'

### 此處檢查輸入雲端連結之大小，在此下載之 zip 檔為 31M

In [2]:
!ls -lh faiss_db.zip

-rw-r--r--@ 1 andrewhsu  staff    31M Apr 14 10:07 faiss_db.zip


### 解壓縮 zip 檔

In [3]:
!unzip faiss_db.zip

Archive:  faiss_db.zip
   creating: faiss_db/
  inflating: faiss_db/index.faiss    
  inflating: faiss_db/index.pkl      


### 1. 安裝並引入必要套件

In [4]:
%pip install -U langchain langchain-community sentence-transformers faiss-cpu gradio openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.4/644.4 kB 2.3 MB/s eta 0:00:00-:--:--
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [6]:
from openai import OpenAI
import gradio as gr

### 2. 自訂 E5 embedding 類別

In [13]:
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

### 3. 載入 解壓縮後的 `faiss_db` 檔案

In [ ]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

### 4. 設定指定之 LLM 模型

In [14]:
import os
from google.colab import userdata

如之前, 我們會用 OpenAI API。這裡使用 Groq 服務, 可改成你要的服務。

In [15]:
api_key = userdata.get('Groq')

In [16]:
os.environ["OPENAI_API_KEY"] = api_key

目前模型和 `base_url` 使用 Groq 所提供之服務

In [17]:
model = "llama3-70b-8192"
base_url="https://api.groq.com/openai/v1" #模型可以改

In [18]:
client = OpenAI(
    base_url=base_url # 使用 OpenAI 本身不需要這段
)

### 5.  設計 `prompt` 來告知模型人設

In [25]:
system_prompt = """
你是一位專業的音樂推薦助手，你會使用用繁體中文回答問題！

主要目標是根據使用者的問題與最近的心情，推薦適合的歌曲。你的推薦會以「心情」作為核心出發點，選擇能夠呼應當下情緒的音樂清單，並且簡單說明每首歌曲為什麼適合。

如果使用者提供了額外的條件（例如指定歌手、發行年份、語言或曲風），你必須在推薦時優先滿足這些條件，並在回答中清楚標註每首歌的相關資訊（歌名、歌手、發行年份等）。如果找不到完全符合條件的歌曲，請誠實回覆並提供最接近的建議。

回覆時請保持語氣友善，像是在和朋友分享音樂，並將推薦結果整理成清單形式，方便使用者快速瀏覽。

預設建議3~5首歌曲，如果使用者有指定數量，請按照需求提供。推薦時記得說明這些歌曲為什麼符合當下心情與使用者提供的條件。

你會使用用繁體中文回答問題！
"""


prompt_template = """
根據下列資料回答問題：
{retrieved_chunks}

輸入問題：{question}

請根據資料內容回覆，若資料不足請包含，系統會盡快更新資料庫內容。
"""

### 6. 使用 RAG 來回應

搜尋與使用者問題相關的資訊，根據我們的 prompt 樣版去讓 LLM 回應。

In [20]:
chat_history = []

def chat_with_rag(user_input):
    global chat_history

    docs = retriever.get_relevant_documents(user_input)
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": final_prompt},
    ]
    )
    answer = response.choices[0].message.content

    chat_history.append((user_input, answer))
    return answer

### 7. 用 Gradio 打造 「🎧 Spotify 音樂播放推薦系統」 web app

In [27]:
with gr.Blocks() as demo:
    gr.Markdown("""
    # 🎧 Spotify 音樂播放推薦系統

    歡迎來到你的專屬音樂推薦助手！
    請輸入你的心情、想聽的曲風，或是其他條件（例如：歌手、發行年份），系統將會根據你的描述，自動從現有播放清單中推薦適合的歌曲清單。

    範例：
    - 現在很放鬆，想聽一些輕鬆的音樂。
    - 心情不太好，有沒有適合的療癒歌曲？
    - 幫我找 2020 年後發行的快樂歌曲，歌手是周杰倫(Jay Chou)。

    趕快輸入你的心情或需求，試試看吧！🎶

    ❗提醒：部分歌手名稱可能以英文存取資料，建議搜尋時中英文皆輸入
    """)

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="請輸入喜歡的歌曲類型或是現在的心情...")

    def respond(message, chat_history_local):
        response = chat_with_rag(message)
        chat_history_local.append((message, response))
        return "", chat_history_local

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)

<ipython-input-27-8e17bc0d0442>:18: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://17cc79510346cb1b52.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://17cc79510346cb1b52.gradio.live
